# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import scipy
import sklearn.preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from surprise import Dataset, Reader
from surprise import accuracy
from surprise import NormalPredictor
from scipy.sparse import csr_matrix, coo_matrix
from implicit.als import AlternatingLeastSquares
from IPython.display import display
from sklearn.metrics.pairwise import cosine_similarity
from catboost import CatBoostClassifier, Pool
from dotenv import load_dotenv
import os
import boto3
from io import BytesIO

/home/mle-user/sprint_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(".env")

True

In [3]:
session = boto3.Session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name="ru-central1",
)

s3 = session.client(
    "s3", 
    endpoint_url=os.getenv("S3_ENDPOINT_URL"))

In [4]:
# Путь к бакету и ключам
BUCKET_NAME = "s3-student-mle-20241219-9c48261c0c"
ITEMS_KEY = "recsys/data/items.parquet"
EVENTS_KEY = "recsys/data/events.parquet"

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [5]:
# Чтение данных о треках
tracks = pd.read_parquet('tracks.parquet')
# Чтение данных о названиях
catalog_names = pd.read_parquet('catalog_names.parquet')
# Чтение данных о взаимодействиях пользователей
interactions = pd.read_parquet('interactions.parquet')

In [ ]:
print(tracks[["track_id", "albums", "artists", "genres"]].sample(5, random_state=52).sort_values("track_id").set_index("track_id").to_string())

In [ ]:
print(catalog_names[["id", "type", "name"]].sample(5, random_state=52).sort_values("id").set_index("id").to_string())

In [ ]:
print(interactions[["user_id", "track_id", "track_seq", "started_at"]].sample(5, random_state=52).sort_values("user_id").set_index(["user_id", "track_id"]).to_string())

In [ ]:
tracks.info()

In [ ]:
catalog_names.info()

In [ ]:
interactions.info()

# Обзор данных

In [ ]:
interactions.head(10)

Проверяем данные, есть ли с ними явные проблемы.

In [ ]:
# Проверка на наличие пустых списков
empty_albums = tracks[tracks['albums'].apply(len) == 0]
empty_artists = tracks[tracks['artists'].apply(len) == 0]
empty_genres = tracks[tracks['genres'].apply(len) == 0]

print(f"Треки с пустыми альбомами: {len(empty_albums)}")
print(f"Треки с пустыми исполнителями: {len(empty_artists)}")
print(f"Треки с пустыми жанрами: {len(empty_genres)}")

In [6]:
tracks['track_id'] = tracks['track_id'].astype('int32')

In [ ]:
# Удаление треков с пустыми списками (или другие действия)
tracks = tracks[tracks['albums'].apply(len) > 0]
tracks = tracks[tracks['artists'].apply(len) > 0]
tracks = tracks[tracks['genres'].apply(len) > 0]

# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

<div class="alert alert-success">
<h2> Комментарий студента</h2>

1) Выводы:
 - Данные разрознены по типам tracs и interactions
 - Есть пустые значения в tracs

2) Действия:
 - Привел все идентификаторы к типу int32.
 - убрал данные с пустыми списками.

</div>

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [ ]:
# Подсчет количества прослушиваний для каждого трека
track_play_counts = interactions['track_id'].value_counts()

# Описание распределения
print(track_play_counts.describe())

Наиболее популярные треки

In [ ]:
# Получение 10 самых популярных треков
most_popular_tracks = track_play_counts.head(10)
print(most_popular_tracks)

Наиболее популярные жанры

In [ ]:
# Объединение таблиц для получения названий жанров
genre_counts = tracks.explode('genres').merge(catalog_names[catalog_names['type'] == 'genre'], left_on='genres', right_on='id', how='left')
genre_counts = genre_counts['name'].value_counts()

In [ ]:
# Получение 10 самых популярных жанров
most_popular_genres = genre_counts.head(10)
print(most_popular_genres)

Треки, которые никто не прослушал

In [ ]:
# Получение треков, которых нет в interactions
unheard_tracks = tracks[~tracks['track_id'].isin(interactions['track_id'])]
print(f"Количество треков, которые никто не прослушал: {len(unheard_tracks)}")

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [7]:
# Создание словарей для быстрого доступа к именам
artist_names = dict(catalog_names[catalog_names['type'] == 'artist'][['id', 'name']].values)
album_names = dict(catalog_names[catalog_names['type'] == 'album'][['id', 'name']].values)
genre_names = dict(catalog_names[catalog_names['type'] == 'genre'][['id', 'name']].values)

In [8]:
# Функция для замены идентификаторов именами
def replace_ids_with_names(ids, names_dict):
    return [names_dict.get(id, 'Unknown') for id in ids]

In [9]:
# Применение функции к столбцам
tracks['artists'] = tracks['artists'].apply(lambda ids: replace_ids_with_names(ids, artist_names))
tracks['albums'] = tracks['albums'].apply(lambda ids: replace_ids_with_names(ids, album_names))
tracks['genres'] = tracks['genres'].apply(lambda ids: replace_ids_with_names(ids, genre_names))

In [10]:
# Переименование столбцов для ясности
tracks.rename(columns={
    'albums': 'album_names',
    'artists': 'artist_names',
    'genres': 'genre_names'
}, inplace=True)

In [11]:
# Проверка на наличие пустых списков
empty_albums = tracks[tracks['album_names'].apply(len) == 0]
empty_artists = tracks[tracks['artist_names'].apply(len) == 0]
empty_genres = tracks[tracks['genre_names'].apply(len) == 0]

print(f"Треки с пустыми альбомами: {len(empty_albums)}")
print(f"Треки с пустыми исполнителями: {len(empty_artists)}")
print(f"Треки с пустыми жанрами: {len(empty_genres)}")

Треки с пустыми альбомами: 18
Треки с пустыми исполнителями: 15369
Треки с пустыми жанрами: 3687


In [12]:
# Функции для замены пустых списков на строки
def replace_empty_albums(album_list):
    return 'Unknown Album' if not album_list else ', '.join(album_list)

def replace_empty_artists(artist_list):
    return 'Unknown Singer' if not artist_list else ', '.join(artist_list)

def replace_empty_genres(genre_list):
    return 'Unknown Genre' if not genre_list else ', '.join(genre_list)

# Применение функций к столбцам
tracks['album_names'] = tracks['album_names'].apply(replace_empty_albums)
tracks['artist_names'] = tracks['artist_names'].apply(replace_empty_artists)
tracks['genre_names'] = tracks['genre_names'].apply(replace_empty_genres)

In [13]:
tracks=tracks.explode("album_names")

In [14]:
tracks=tracks.explode("artist_names")

In [15]:
tracks=tracks.explode("genre_names")

In [16]:
tracks.head(10)

,track_id,album_names,artist_names,genre_names
0,26,"Taller Children, Taller Children",Elizabeth & the Catapult,"pop, folk"
1,38,"Taller Children, Taller Children",Elizabeth & the Catapult,"pop, folk"
2,135,"Wild Young Hearts, Wild Young Hearts, Wild You...",Noisettes,pop
3,136,"Wild Young Hearts, Wild Young Hearts, Wild You...",Noisettes,pop
4,138,"Wild Young Hearts, Wild Young Hearts, Don't Up...",Noisettes,pop
5,139,"Wild Young Hearts, Wild Young Hearts, Wild You...",Noisettes,pop
6,140,"Wild Young Hearts, Wild Young Hearts, Wild You...",Noisettes,pop
7,141,"Wild Young Hearts, Wild Young Hearts, I Am Wor...",Noisettes,pop
8,143,"Wild Young Hearts, Wild Young Hearts, Wild You...",Noisettes,pop
9,144,"Lonesome Crow, Lonesome Crow",Scorpions,"hardrock, allrock"


In [ ]:
tracks = tracks.drop_duplicates()

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [17]:
tracks.to_parquet(ITEMS_KEY)
interactions.to_parquet(EVENTS_KEY)

In [ ]:
# Сохранение данных о треках в BytesIO
buffer_items = BytesIO()
tracks.to_parquet(buffer_items, engine='pyarrow')
buffer_items.seek(0)

# Загрузка данных о треках в S3
s3.put_object(
    Bucket=BUCKET_NAME,
    Key=ITEMS_KEY,
    Body=buffer_items.getvalue()
)

In [ ]:
# Сохранение взаимодействий в BytesIO
buffer_events = BytesIO()
interactions.to_parquet(buffer_events, engine='pyarrow')
buffer_events.seek(0)

# Загрузка данных о взаимодействиях в S3
s3.put_object(
    Bucket=BUCKET_NAME,
    Key=EVENTS_KEY,
    Body=buffer_events.getvalue()
)

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

In [18]:
# Удаление ненужных переменных
del interactions
del catalog_names
del tracks

# Принудительная сборка мусора
import gc
gc.collect()

0

### Шаги для перезапуска ядра:

1. Перейдите в меню Jupyter Notebook: Kernel -> Restart Kernel.

2. Выберите опцию Restart and clear output, если хотите очистить все выводы ячеек.

3. После перезапуска ядра выполните секцию Инициализация и продолжите с этапа 3.

- После перезапуска ядра выполните необходимые начальные секции, которые настраивают сессию boto3 и загружают необходимые данные из S3:

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

In [ ]:
# Загрузка items.parquet
response = s3.get_object(Bucket=BUCKET_NAME, Key=ITEMS_KEY)
buffer_items = BytesIO(response['Body'].read())
items = pd.read_parquet(buffer_items)

In [ ]:
# Загрузка events.parquet
response = s3.get_object(Bucket=BUCKET_NAME, Key=EVENTS_KEY)
buffer_events = BytesIO(response['Body'].read())
events = pd.read_parquet(buffer_events)

In [19]:
# Чтение данных о треках
items = pd.read_parquet('recsys/data/items.parquet')
# Чтение данных о названиях
events = pd.read_parquet('recsys/data/events.parquet')

In [ ]:
# Принудительная сборка мусора
import gc
gc.collect()

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

In [20]:
# Преобразуем столбец started_at в datetime
events['started_at'] = pd.to_datetime(events['started_at'])

In [21]:
from datetime import datetime

In [22]:
# Разделение данных
train_cutoff_date = datetime(2022, 12, 16)
train_data = events[events['started_at'] < train_cutoff_date]
test_data = events[events['started_at'] >= train_cutoff_date]

# Топ популярных

Рассчитаем рекомендации как топ популярных.

In [23]:
# Подсчет количества прослушиваний для каждого трека
track_popularity = train_data['track_id'].value_counts().reset_index()
track_popularity.columns = ['track_id', 'popularity']
top_popular = track_popularity.head(100)  # Топ 100 самых популярных треков

# Сохранение в файл
top_popular.to_parquet('recsys/recommendations/top_popular.parquet')

# Персональные

Рассчитаем персональные рекомендации.

In [25]:
# Перекодируем идентификаторы пользователей:
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
train_data["user_id_enc"] = user_encoder.transform(train_data["user_id"])
test_data["user_id_enc"] = user_encoder.transform(test_data["user_id"])


/tmp/ipykernel_2032/2165694531.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["user_id_enc"] = user_encoder.transform(train_data["user_id"])
/tmp/ipykernel_2032/2165694531.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["user_id_enc"] = user_encoder.transform(test_data["user_id"])


In [26]:
# Перекодируем идентификаторы объектов:
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["track_id"])
items["track_id_enc"] = item_encoder.transform(items["track_id"])
train_data["track_id_enc"] = item_encoder.transform(train_data["track_id"])
test_data["track_id_enc"] = item_encoder.transform(test_data["track_id"])

/tmp/ipykernel_2032/3620722601.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["track_id_enc"] = item_encoder.transform(train_data["track_id"])
/tmp/ipykernel_2032/3620722601.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["track_id_enc"] = item_encoder.transform(test_data["track_id"])


# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.